# **C. Ingredient Optimisation**

<span style="font-size: 14px;"><b>1.What are the standard ingredients for each pizza?</b></span>

- STRING\_TO\_ARRAY() function : This function is used to split a string into array elements using supplied delimiter and optional null string.
- Cast a text to an integer using CAST()
- UNNEST() function : This function is used to expand an array to a set of rows.
- count the toppings, if it is used in both the pizzas it will return count as 2 then that toppings will be the standard ingredients for each pizza

In [1]:
with pizza_recipes_cte as(
	SELECT
		pizza_id,
		CAST(
			UNNEST(string_to_array(toppings, ', ')) AS INT
			) AS topping_id
	FROM pizza_recipes)
SELECT
	pt.topping_name,
	COUNT(*)
FROM pizza_recipes_cte pr
JOIN pizza_toppings pt
	ON pr.topping_id=pt.topping_id
	GROUP BY pt.topping_name
	HAVING COUNT(*)=2

(2 row(s) affected)

Total execution time: 00:00:00.009

topping_name,count
Mushrooms,2
Cheese,2


<span style="font-size: 14px;"><b>2. What was the most commonly added extra?</b></span>

- <span style="font-size: 14px;">where extras is not null</span>
- <span style="font-size: 14px;">STRING_TO_ARRAY() to split a string into array elements using supplied delimiter(,)</span>
- <span style="font-size: 14px;">UNNEST() function : This function is used to expand an array to a set of rows.(same as above solution)</span>
- <span style="font-size: 14px;">count the toppings&nbsp;</span> 
- <span style="font-size: 14px;">sort the count in descinding order</span>
- <span style="font-size: 14px;">limit 1 (beacause we only want one item)</span>

In [2]:
WITH extras_cte as(
	SELECT
		CAST(
     		UNNEST(string_to_array(extras, ', ')) AS INT
			) AS extras_id
	FROM new_customer_orders
	WHERE extras is not null)
SELECT
	pt.topping_name,
	count(*)
FROM extras_cte as ec
JOIN pizza_toppings pt
	ON ec.extras_id = pt.topping_id
	GROUP BY pt.topping_name
	ORDER BY count(*) DESC
	LIMIT 1;

(1 row(s) affected)

Total execution time: 00:00:00.004

topping_name,count
Bacon,4


<span style="font-size: 14px;"><b>3.What was the most common exclusion?</b></span>

-  same as above solution but excluiosn instead of extras

In [4]:
WITH exclusions_cte as(
	SELECT
		CAST(
			UNNEST(string_to_array(exclusions, ', ')) AS INT
			) AS exclusions_id
	FROM new_customer_orders
	WHERE exclusions is not null)
SELECT 
	pt.topping_name,
	COUNT(*)
FROM exclusions_cte as ec
JOIN pizza_toppings pt
	ON ec.exclusions_id = pt.topping_id
	GROUP BY pt.topping_name
	ORDER BY count(*) DESC
	LIMIT 1;

(1 row(s) affected)

Total execution time: 00:00:00.003

topping_name,count
Cheese,4


<span style="font-size: 14px;"><b>4. Generate an order item for each record in the customers_orders table in the format of one of the following:</b></span>

<span style="font-size: 14px;"><b><span style="white-space:pre"></span>-- Meat Lovers</b></span>

<span style="font-size: 14px;"><b><span style="white-space:pre"></span>-- Meat Lovers - Exclude Beef</b></span>

<span style="font-size: 14px;"><b><span style="white-space:pre"></span>-- Meat Lovers - Extra Bacon</b></span>

<span style="font-size: 14px;"><b><span style="white-space:pre"></span>-- Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers</b></span>

In [7]:
CREATE VIEW temp_table as(
with cte2 as(
with cte1 as(
	SELECT 
	c.order_id,
	c.pizza_id,
	SUBSTR(c.exclusions,1,1) as exclusion1,
	SUBSTR(c.exclusions,4,1) as exclusion2,
	SUBSTR(c.extras,1,1) as extra1,
	SUBSTR(c.extras,4,1) as extra2,
	pn.pizza_name
FROM new_customer_orders c
JOIN pizza_names pn
	ON c.pizza_id = pn.pizza_id
	WHERE pn.pizza_name ='Meatlovers')
	SELECT 
		order_id,
		pizza_id,
		CAST((CASE
			WHEN exclusion1 = '' THEN NULL
            WHEN exclusion1 = 'null' THEN NULL
            ELSE exclusion1
        END)AS INT) AS exclusion1,
		CAST((CASE
			WHEN exclusion2 = '' THEN NULL
            WHEN exclusion2 = 'null' THEN NULL
            ELSE exclusion2
        END)AS INT) AS exclusion2,
		CAST((CASE
			WHEN extra1 = '' THEN NULL
            WHEN extra1 = 'null' THEN NULL
            ELSE extra1
        END)AS INT) AS extra1,
		CAST((CASE
			WHEN extra2 = '' THEN NULL
            WHEN extra2 = 'null' THEN NULL
            ELSE extra2
        END)AS INT) AS extra2,
		pizza_name
	FROM cte1)
	SELECT 
		order_id,
		pizza_id,
		exclusion1,
		exclusion2,
		extra1,
		extra2,
		pizza_name,
		pt1.topping_name as exclusion1_name,
		pt2.topping_name as exclusion2_name,
		pt3.topping_name as extra1_name,
		pt4.topping_name as extra2_name
	FROM cte2
	LEFT JOIN pizza_toppings pt1
		ON cte2.exclusion1= pt1.topping_id
	LEFT JOIN pizza_toppings pt2
		ON cte2.exclusion2=pt2.topping_id
	LEFT JOIN pizza_toppings pt3
		ON cte2.extra1=pt3.topping_id
	LEFT JOIN pizza_toppings pt4
		ON cte2.extra2=pt4.topping_id)

Commands completed successfully

Total execution time: 00:00:00.010

In [8]:
SELECT 
	order_id,
	pizza_name,
	exclusion1_name,
	exclusion2_name,
	extra1_name,
	extra2_name,	
	CASE
		WHEN exclusion1_name is not null and exclusion2_name is not null and extra1_name is not null and extra2_name is not null 
		THEN CONCAT(pizza_name,' - Exclude ',exclusion1_name,', ',exclusion2_name,' - Extra ',extra1_name,', ',extra2_name)
		WHEN exclusion1_name is not null and extra1_name is not null
		THEN CONCAT(pizza_name,' - Exclude ',exclusion1_name,', ',exclusion2_name,' - Extra ',extra1_name,', ',extra2_name)
		WHEN exclusion1_name is not null
		THEN CONCAT(pizza_name,' - Exclude ',exclusion1_name)
		WHEN extra1_name is not null
		THEN CONCAT(pizza_name,' - Extra ',extra1_name)
		ELSE pizza_name
	END as status	
FROM temp_table
ORDER BY order_id

(10 row(s) affected)

Total execution time: 00:00:00.010

order_id,pizza_name,exclusion1_name,exclusion2_name,extra1_name,extra2_name,status
1,Meatlovers,NULL,NULL,NULL,NULL,Meatlovers
2,Meatlovers,NULL,NULL,NULL,NULL,Meatlovers
3,Meatlovers,NULL,NULL,NULL,NULL,Meatlovers
4,Meatlovers,Cheese,NULL,NULL,NULL,Meatlovers - Exclude Cheese
4,Meatlovers,Cheese,NULL,NULL,NULL,Meatlovers - Exclude Cheese
5,Meatlovers,NULL,NULL,Bacon,NULL,Meatlovers - Extra Bacon
8,Meatlovers,NULL,NULL,NULL,NULL,Meatlovers
9,Meatlovers,Cheese,NULL,Bacon,Chicken,"Meatlovers - Exclude Cheese, - Extra Bacon, Chicken"
10,Meatlovers,NULL,NULL,NULL,NULL,Meatlovers
10,Meatlovers,BBQ Sauce,Mushrooms,Bacon,Cheese,"Meatlovers - Exclude BBQ Sauce, Mushrooms - Extra Bacon, Cheese"


**5.\*\*Generate an alphabetically ordered comma separated ingredient list for each pizza order from the `customer_orders` table and add a `2x` in front of any relevant ingredients\*\***

- **For example: `"Meat Lovers: 2xBacon, Beef, ... , Salami"`**

**6\. What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?**  

- <span style="font-size: 14px;">join new_customer_orders and new_runner_orders using order_id</span>
- <span style="font-size: 14px;">choose delivered pizzas (cancellation is null)</span>
- <span style="font-size: 14px;">then join pizza_names to get pizza name</span>

In [9]:
with cte as(SELECT 
	c.order_id,
	c.customer_id,
	c.pizza_id,
	CAST(
     UNNEST(string_to_array(pr.toppings, ', ')) AS INT
     ) AS topping_id
FROM new_customer_orders c
JOIN new_runner_orders r
	on c.order_id = r.order_id
JOIN pizza_recipes pr
	ON c.pizza_id = pr.pizza_id
	WHERE r.cancellation is null
	)
	SELECT
		pt.topping_name,
		cte.topping_id,
		count(*)
	FROM cte 
	JOIN pizza_toppings pt
		ON cte.topping_id = pt.topping_id
	group by cte.topping_id,pt.topping_name
	order by count(*) desc

(12 row(s) affected)

Total execution time: 00:00:00.003

topping_name,topping_id,count
Cheese,4,12
Mushrooms,6,12
Chicken,5,9
Pepperoni,8,9
Salami,10,9
Bacon,1,9
BBQ Sauce,2,9
Beef,3,9
Peppers,9,3
Tomatoes,11,3
